# 7.1　Keras Functional API



## 7.1.1　Introduction

Functional APIでは、tensorを直接操作し、tensorを受け取ってtensorを返す関数(function)として層を使用する。

それによりFunctional APIと呼ばれる。



In [1]:
from keras import Input, layers

# テンソル
input_tensor = Input(shape=(32,))

# 層は関数
dense = layers.Dense(32, activation='relu')

# テンソルで呼び出された層はテンソルを返す
output_tensor = dense(input_tensor)

Using TensorFlow backend.


簡単な例

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# おなじみのSequential　Model
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Functional APIでそれに相当するもの
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# Modelクラスは入力テンソルと出力テンソルをモデルに変換する
model = Model(input_tensor, output_tensor)

# このモデルのアーキテクチャを確認
seq_model.summary()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
___________________________________

入力テンソルと出力テンソルだけを使ってModelオブジェクトをインスタンス化している。Kerasは、input_tensorからoutput_tensorまでの間にある層を全て取得し、それらをグラフ形式のデータ構造、つまりModelにまとめる。Modelのインスタンスのcompile, fit, evaluateに関してはFunctional APIはSequential modelと同じ。

In [3]:
# モデルをコンパイル
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 訓練に使用するダミーのNumpyデータを生成
import numpy as np 
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# モデルを１０エポックで訓練
model.fit(x_train, y_train, epochs=10, batch_size=128)

# モデルを評価
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 12.1669
Epoch 2/10
1000/1000 [==============================] - 0s 33us/step - loss: 13.6293
Epoch 3/10
1000/1000 [==============================] - 0s 30us/step - loss: 15.9327
Epoch 4/10
1000/1000 [==============================] - 0s 34us/step - loss: 18.9705
Epoch 5/10
1000/1000 [==============================] - 0s 34us/step - loss: 22.0116
Epoch 6/10
1000/1000 [==============================] - 0s 32us/step - loss: 25.1298
Epoch 7/10
1000/1000 [==============================] - 0s 37us/step - loss: 28.4055
Epoch 8/10
1000/1000 [==============================] - 0s 30us/step - loss: 31.7187
Epoch 9/10
1000/1000 [==============================] - 0s 36us/step - loss: 35.4749
Epoch 10/10
1000/1000 [==============================] - 0s 95us/step


## 7.1.2　多入力モデル

・複数の入力を持つモデルの構築が可能


・一般にはkeras.layers.addやkeras.layers.concatenateで入力分岐をマージする

### list 7-1: 2 input

・2つの入力を持つ質問応答モデルの実装

In [6]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# テキスト入力は整数の可変長のシーケンス
# なお、必要であれば、入力に名前をつけることもできる
text_input = Input(shape=(None,), dtype='int32', name='text')

# 入力をサイズが６４のベクトルシーケンスに埋め込む
embedded_text = layers.Embedding(text_vocabulary_size)(text_input, 64)

# LSTMを通じてこれらのベクトルを単一のベクトルにエンコード
encorded_text = layers.LSTM(32)(embedded_text)

# 質問入力でも（異なる層のインスタンスを使って）同じプロセスを繰り返す
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# エンコードされたテキストと質問を連結
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

#　ソフトマックス分類器を追加
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenate)

# モデルをインスタンス化するときには、2つの入力と1つの出力を指定
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])


TypeError: __init__() missing 1 required positional argument: 'output_dim'

### list 7-2: data for multi-input model

In [ ]:
import numpy as np
num_samples = 1000
max_length = 100

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# ダミーのNumpyデータを生成
text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, 
                             size=(num_samples, max_length))

#　答えに（整数ではなく）one-hotエンコーディングを適用
answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size = num_samples)
for i, x in enumerate(answers):
  x[indices[i]] = 1

#　入力リストを使った適合
model.fit([text, question], answers, epochs=10, batch_size=128)

#　入力ディクショナリを使った適合（入力に名前を付ける場合のみ）
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

## 7.1.3　多出力モデル

### list 7-3：3 output

Functional APIを使って複数の出力を持つモデルを構築できる。例えば、同一の匿名ユーザによるSNSの一連の投稿を入力として、そのユーザーの年齢、性別、所得水準といった複数の属性を予測するネットワークが考えられる。

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

#　出力層に名前が付いていることに注意
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])                                

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

### list 7-4： compile option (multi-loss)



In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse',
                    'categorical_crossentropy',
                    'binary_crossentropy'])

#　上記と同じ（出力層に名前をつけている場合にのみ可能）
model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

### list 7-5： compile option (loss_weights)

損失値の貢献度がかなり不均衡である場合、損失値に対して重要度を割り当てる。特に損失値の尺度が異なる場合、例えば年齢回帰タスクに使用されるMSEは一般に3〜5の値をとるが、性別分類タスクに使用される交差エントロピーの値は最低でも0.1になることがある。よって重要度のバランスをとるため、MSEに0.25の重みを、交差エントロピーに10の重みを割り当てることができる。

In [5]:
model.compile(optimizer='rmsprop',
              loss=['mse', 
                    'categorical_crossentropy',
                    'binary_crossentropy'],
              loss_weights=[0.25, 
                            1., 
                            10.])

#　上記と同じ（出力層に名前をつけている場合にのみ可能）
model.compile(optimizer='rmsprop',
              loss={'age':'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age':0.25, 
                            'income':1., 
                            'gender':10.})

ValueError: When passing a list as loss, it should have one entry per model outputs. The model has 1 outputs, but you passed loss=['mse', 'categorical_crossentropy', 'binary_crossentropy']

### list 7-6： data for multi-output model

In [ ]:
#　age_targets,　income_targets, gender_targetsはNumPy配列と仮定
model.fit(posts, [age_targets,　income_targets, gender_targets],
           epochs=10, batch_size=64)

#　上記と同じ（出力層に名前をつけている場合にのみ可能）
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

## 7.1.4　層の有向非巡回グラフ

・Directed Acyclic Graph

・Inception Module







### **WHY 1x1 convolution?**


畳み込みでは、入力テンソルの全てのタイルのまわりで空間パッチが抽出され、全てのパッチに同じ変換が適用されます。エッジケースは、抽出されたパッチがたった1つのタイルで構成されている場合です。その場合、畳み込み演算はDense層を通じて各タイルベクトルを実行することに等しくなります。つまり、入力テンソルの各チャネルの情報を混ぜ合わせた特徴量を計算することになりますが、（一度に調べるタイルは2つだけなので）空間にまたがって情報を混ぜ合わせることはありません。Inceptionモジュールでは、そうした１x１の畳み込みが挿入され、**チャネルごとの表現学習と空間ごとの表現学習の因数分解に貢献します**。１x１の畳み込みは、pw畳み込み（pointwise convolution）とも呼ばれます。このオプションは、各チャネルが空間を跨いで強い自己相関関係にある場合には合理的なのですが、チャネルによっては相互の関連性が低いことも考えられます。



**Inceptionモジュール**　（４次元の入力テンソルxが存在する前提）

In [ ]:
from keras import layers
from keras import Input
import numpy as np

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)


**残差接続**(４次元の入力テンソルxが存在する前提)

In [ ]:
# 特徴マップのサイズが同じ場合

x = .....

# xに変換を適用
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

# 元のxを出力特徴量に追加
y = layers.add([y, x])


In [ ]:
# 特徴マップのサイズが異なっている場合に、線形残差接続を使用する実装

x = .....

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# 元のテンソルxをyと同じ形状にするための１x１畳み込みを使った線形ダウンサンプリング
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

# 残差テンソルを出力特徴量に追加
y = layers.add([y, residual])

### **DLの表現のBottleneck**

Sequentialモデルでは、連続する表現層はそれぞれ前の層の上に構築されます。つまり、その層からアクセスできる情報は、前の層の活性化に含まれているものだけです。1つの層が小さすぎる場合（特徴量の次元の数が少なすぎるなど）、モデルはその層の活性化に詰め込むことができる情報の量によって制限されることになります。

この概念を理解するために、信号処理についての考えてみましょう。一連の演算で構成された音声処理のパイプラインがあり、それぞれの演算の入力は1つ前の演算の出力です。ある演算が信号を低周波域（0〜15kHzなど）にクロッピングした場合、下流の演算は削除された周波数を復元できなくなります。情報の損失は全て恒久的です。残差接続は、上流の情報を下流に再注入することで、ディープラーニングモデルのこの問題を部分的に解決します。

### **DLでの勾配消失問題**

バックプロパゲーション（誤差逆伝播法）は、ディープニューラルネットワークの訓練に使用されるマスターアルゴリズムです。このアルゴリズムは、出力側の損失関数からのフィードバック信号を入力側の層へ伝播するという仕組みになっています。このフィードバック信号を伝播させる層が深いスタックになっている場合、フィードバック信号が弱められたり完全に失われたりして、ネットワークを訓練できなくなる可能性があります。この問題は**勾配消失**（vanishing gradient）と呼ばれます。

この問題が発生するのは、ディープニューラルネットワーク（DNN）と、非常に長いシーケンスを処理するリカレントニューラルネットワーク（RNN）です。どちらの場合もフィードバック信号は演算の長い連鎖に渡って伝播されなければなりません。既に説明したように、RNNではLSTMを使ってこの問題に対処します。それにより、メインの処理トラックに情報を同時に運び込むキャリートラックが追加されます。フィードフォワードDNNでは、残差接続が同じような役割を果たしますが、この場合はさらに単純です。残差接続により、スタックの深さに関係なく勾配を伝播させるのに役立つ、完全に線形のキャリートラックが追加されるからです。

## 7.1.5　層の重みの共有

Siamese Network

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

# 単一のLSTM層を一度だけインスタンス化
lstm = layers.LSTM(32)

# モデルの左側の分岐を構築
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# モデルの右側の分岐を構築
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 最後に分類器を構築
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# モデルのインスタンス化と訓練
# このようなモデルを訓練する時には、LSTM層の重みが両方の入力に基づいて更新される
model = Model([left_input, right_input], predictions)
model.fit([left_input, right_input], targets)

## 7.1.6　層としてのモデル

・層を使用するときのようにモデルを使用できる。

・モデルをより大きな層として考えることができる。

・入力テンソルを使ってモデルを呼び出し、出力テンソルを取得することが可能


入力テンソルを使ってモデルを呼び出し、出力テンソルを取得することが可能
```
y = model(x)
```

入力テンソルと出力テンソルが複数の場合

```
y1, y2 = model([x1, x2])
  ```



In [ ]:
from keras import layers
from keras import applications
from keras import Input

# ベースとなる画像処理モデルはXceptionネットワーク（畳み込みベースのみ）
xception_base = applications.Xception(weights=None, include_top=False)

# 入力は250x250のRGB画像
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 同じビジョンモデルを２回呼び出す
left_features = xception_base(left_input)
right_input = xception_base(right_input)

# マージ後の特徴量には、右の視覚フィードと左の視覚フィードの情報が含まれている
merged_features = layers.concatenate([left_features, right_input], axis=-1)

## 7.1.7　まとめ




・層の線形スタック以外のものが必要な場合に使用する方法。

・複数の入出力、複雑な内部ネットワークトポロジを持つモデルを構築する方法。

・同じ層またはモデルのインスタンスを複数回呼び出すことで、様々な処理分岐にまたがって層やモデルの重みを再利用する方法。